# Parsing Jupyter Notebooks

In [1]:
import nbformat
import io

# Reading Notebooks

In [2]:
def read_nb(nb):
    with io.open(nb, 'r') as f:
        nb = nbformat.read(f, nbformat.NO_CONVERT)
    return nb

In [3]:
source_nb = 'atoms/visualization/choropleth_classification.ipynb'

inb = read_nb(source_nb)

In [4]:
type(inb)

nbformat.notebooknode.NotebookNode

In [5]:
inb.keys()

dict_keys(['metadata', 'cells', 'nbformat', 'nbformat_minor'])

In [6]:
inb.metadata

{'anaconda-cloud': {},
 'kernelspec': {'display_name': 'Python [Root]',
  'language': 'python',
  'name': 'Python [Root]'},
 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
  'file_extension': '.py',
  'mimetype': 'text/x-python',
  'name': 'python',
  'nbconvert_exporter': 'python',
  'pygments_lexer': 'ipython3',
  'version': '3.5.2'}}

In [7]:
cells = inb['cells']

In [8]:
type(cells)

list

In [9]:
len(cells)

99

In [10]:
type(cells)

list

## Let's replace the cells in the in-memory notebook to create a new one

In [11]:
# extract only ever other cell
new_cells = [ c for i,c in enumerate(cells) if i%2]

In [12]:
len(new_cells)

49

In [13]:
inb['cells'] = new_cells

In [14]:
with io.open('smaller.ipynb', 'w', encoding='utf8') as f:
    nbformat.write(inb, f, nbformat.NO_CONVERT)

In [15]:
snb = read_nb('smaller.ipynb')

In [16]:
len(snb.cells)

49

## Notebook Cells

In [17]:
c0 = snb.cells[0]

In [18]:
type(c0)

nbformat.notebooknode.NotebookNode

In [19]:
c0.keys()

dict_keys(['cell_type', 'metadata', 'source'])

In [20]:
c0['cell_type']

'markdown'

In [21]:
c0['source']

'## Introduction\n\n* what is classification\n* role in choropleth mapping\n* explore classification using PySAL'

In [22]:
c0['metadata']

{'collapsed': True}

## Types of cells in  this notebook

In [23]:
from collections import defaultdict
def get_structure(cells):
    cell_types = defaultdict(list)
    for i, cell in enumerate(cells):
        cell_types[cell.cell_type].append(i)
    return cell_types
            

In [24]:
cell_types = get_structure(snb.cells)

In [25]:
cell_types.keys()

dict_keys(['code', 'markdown'])

In [26]:
for ct, cells in cell_types.items():
    print('Cell Type: %s\t %d cells'% (ct, len(cells)))

Cell Type: code	 38 cells
Cell Type: markdown	 11 cells


In [27]:
code_cell_idx = cell_types['code'][0]
code_cell_idx

2

In [28]:
snb.cells[code_cell_idx]

{'cell_type': 'code',
 'execution_count': 3,
 'metadata': {'collapsed': False},
 'outputs': [{'data': {'text/plain': '{\'description\': \'Mexican states regional income 1940-2000\',\n \'explanation\': [\'Data used in   Rey, S.J. and M.L.  Sastre Gutierrez. (2010) "Interregional inequality\',\n  \'dynamics in Mexico." Spatial Economic Analysis, 5: 277-298\',\n  \'* mexico.csv: attribute data\',\n  \'* mexico.gal: spatial weights in GAL format\',\n  \'Polygon data, n=32, k=13\'],\n \'name\': \'mexico\'}'},
   'execution_count': 3,
   'metadata': {},
   'output_type': 'execute_result'}],
 'source': "ps.examples.explain('mexico')"}

In [29]:
mkd_cell_idx = cell_types['markdown'][0]
mkd_cell_idx

0

In [30]:
snb.cells[mkd_cell_idx]

{'cell_type': 'markdown',
 'metadata': {'collapsed': True},
 'source': '## Introduction\n\n* what is classification\n* role in choropleth mapping\n* explore classification using PySAL'}

## Turning Output Cells OFF

In [31]:
def remove_outputs(nb):
    """Set output attribute of all code cells to be empty"""
    for cell in nb.cells:
        if cell.cell_type == 'code':
            cell.outputs = []

def clear_notebook(old_ipynb, new_ipynb):
    with io.open(old_ipynb, 'r') as f:
        nb = nbformat.read(f, nbformat.NO_CONVERT)

    remove_outputs(nb)
    
    with io.open(new_ipynb, 'w', encoding='utf8') as f:
        nbformat.write(nb, f, nbformat.NO_CONVERT)

source_nb = 'atoms/visualization/choropleth_classification.ipynb'

new_nb = 'nout.ipynb'
clear_notebook(source_nb, new_nb)